In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [13]:
import pickle

import numpy as np

import chemistry
from pprint import pprint

In [50]:
with open('../data/molecules.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [51]:
from chemistry import Molecule, atomic_number

def enhance_molecule(m):
    mnew = Molecule.copy(m)
    
    atom_weights = np.array([chemistry.atomic_number[a] for a in m.symbols], dtype='float32')
    mnew.field_intensity = np.empty(m.n_atoms, dtype='float32')
    
    for i in range(1):
        pos_this = np.tile(m.positions[i], (m.n_atoms, 1))
        pos_other = m.positions
        diff = pos_this - pos_other
        dist2 = np.sum(diff * diff, axis=1)
        
        fi_per_atom = atom_weights
        sel = np.arange(m.n_atoms) != i
        fi_per_atom[sel] /= dist2[sel]
        fi_per_atom[i] = 0
        
        mnew.field_intensity[i] = fi_per_atom.sum()
        
    return mnew

    
enhance_molecule(molecules['dsgdb9nsd_000001'])

Name: dsgdb9nsd_000001
Atoms:
  C 0: [-0.01269814  1.0858041   0.008001  ]
  H 1: [ 0.00215042 -0.00603132  0.00197612]
  H 2: [1.0117308e+00 1.4637512e+00 2.7657481e-04]
  H 3: [-0.54081506  1.4475266  -0.8766437 ]
  H 4: [-0.5238136  1.4379326  0.9063973]
Bonds:
  C(0) - H(1)
  C(0) - H(2)
  C(0) - H(3)
  C(0) - H(4)
Field intensity :
  C(0) - 3.354708433151245
  H(1) - 1.1923612356185913
  H(2) - 1.192358374595642
  H(3) - 1.1923468112945557
  H(4) - 1.1923494338989258


In [52]:
molecules_enh = {mn:enhance_molecule(molecules[mn]) for mn in molecules}

In [53]:
with open('../data/molecules_enh.pickle', 'wb') as f:
    pickle.dump(molecules_enh, f)